## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import pandas as pd
import seaborn as sns
from dotenv import load_dotenv
from vidore_benchmark.evaluation.eval_manager import EvalManager
from vidore_benchmark.utils.constants import OUTPUT_DIR

RESULTS_DIR = OUTPUT_DIR / "token_pooling"
RESULTS_DIR.mkdir(exist_ok=True, parents=True)

load_dotenv(override=True)
sns.set_style("whitegrid")

while "experiments" not in os.listdir():
    os.chdir("..")

In [ ]:
eval_managers = [
    EvalManager.from_dir("experiments/2024-08-06_impact_of_pool_factor_on_retrieval/outputs/docvqa/metrics/"),
    EvalManager.from_dir("experiments/2024-08-06_impact_of_pool_factor_on_retrieval/outputs/energy/metrics/"),
    EvalManager.from_dir("experiments/2024-08-06_impact_of_pool_factor_on_retrieval/outputs/shift_project/metrics/"),
]
list_df = [eval_manager.melted for eval_manager in eval_managers]

df = pd.concat(list_df, ignore_index=True)
df

In [ ]:
df["pool_factor"] = df["model"].str.extract(r"pool_factor_(\d+)").astype(int)
df["model"] = df["model"].str.extract(r"(.*)_metrics_pool_factor_\d+")

df = df[df["metric"] == "ndcg_at_5"]
df = df[df["pool_factor"] <= 10]

df

In [ ]:
sns.lineplot(data=df, x="pool_factor", y="score", hue="dataset")